# 3장. 다항 선형 회귀와 정규화 기법

---

## 학습 목표
- 정규화 기법이 왜 필요한지를 이해합니다.
- 다항 선형 회귀에서 사용되는 릿지, 라쏘, 엘라스틱 정규화 기법에 대해서 학습합니다.

---

## 목차

### 1. 정규화 기법 개념
1. 선형 회귀의 정규화 기법

### 2. 릿지 정규화 기법
1. Loss 함수와 규제항
2. 규제항 하이퍼 파라미터

### 3. 라쏘, 엘라스틱 넷 정규화 기법
1. 라쏘 정규화
2. 엘라스틱 넷 정규화


---

## 3. 라쏘, 엘라스틱 넷 정규화 기법

### 3-1. 라쏘 정규화

라쏘(Lasso) 정규화는 릿지에서 원형 규제항을 더한 것과는 달리 아래와 같이 선형 파라미터의 절대값을 더한 규제항을 추가하여 정의합니다.

##### 라쏘 loss 함수

> $$Loss(\mathbf{w}) = MSE(\mathbf{w})+\frac{\lambda}{N} \sum_{i=0}^{p}|w_{i}|$$

규제항의 형태가 변하였기에 규제항의 크기도 릿지와는 다르게 계산됩니다.

아래 그림과 같이 같은 크기를 갖는 규제항은 다이아몬드 형태의 그래프로 그려집니다.

<img src="img/3-3-1.png" width="40%" height="40%" title="las1" alt="las1"></img>

**scikit-learn을 사용한 라쏘 정규화**

scikit-learn에서는 라쏘 정규화는 기존에 사용하던 `LinearRegression` class를 사용하여 수행할 수 있습니다.

다만 class는 초기화 시, `.Lasso(alpha)`을 사용하여 $\lambda$ 값을 의미하는 `alpha`값을 설정하여야 합니다.

##### <예제 1> 라쏘 정규화

`2-2. <예제 4>` 에서와 같은 데이터를 사용하여 라쏘 정규화를 수행하고 loss 값을 출력해 봅시다.

In [4]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

# 학습용 데이터
feature_data = np.array([1, 2, 3, 4]).reshape((-1,1))
label_data = np.array([3.1,4.9,7.2,8.9]).reshape((-1,1))

# 4차항 선형 회귀에 해당되는 4를 초기값으로 입력
poly = PolynomialFeatures(4)

# fit 함수를 사용하여 X로 변환 작업을 수행
X = poly.fit_transform(feature_data)
print("X: \n{}\n".format(X))

# 라쏘 모델 설정 알파 값을 바꿔가며 수행
lasso_model = linear_model.Lasso(alpha=1)

# 학습 수행
lasso_model.fit(X, label_data)

# scikit-learn 에서는 loss 함수가 모델안에 내정되어 있지 않기에 정의
def loss(prediction, label):
        
    error = label - prediction
    ls = np.mean(error**2)

    return ls

print("loss: {}\n".format(loss(lasso_model.predict(X), label_data))) 

X: 
[[  1.   1.   1.   1.   1.]
 [  1.   2.   4.   8.  16.]
 [  1.   3.   9.  27.  81.]
 [  1.   4.  16.  64. 256.]]

loss: 9.08835766581683



위 실습에서 모델의 $\lambda$ 값을 의미하는 `alpha`값을 바꿔가며 loss 값이 어떻게 바뀌는지 수행해 봅시다.

---

### 3-2. 엘라스틱 넷 정규화

엘라스틱 넷(Elastic net)은 라쏘와 릿지의 선형 결합으로 아래의 수식으로 정의 됩니다.

##### 엘라스틱 넷 loss 함수

> $$Loss(\mathbf{w}) = MSE(\mathbf{w})+\frac{\lambda}{N} ((1-\alpha)\sum_{i=0}^{p}w_{i}^{2}+\alpha\sum_{i=0}^{p}|w_{i}|)$$

$\alpha$는 0보다 크며 1보다 작은 값으로 라쏘와 릿지의 비율을 의미합니다.

따라서 그래프도 다이아몬드와 원형이 일정 비율로 합쳐진 형태를 취합니다.

<img src="img/3-3-2.png" width="50%" height="50%" title="ela1" alt="ela1"></img>

정규화 수행시 어떤 기법을 써야 좋은지는 데이터의 분포와 성격에 따라 다르며 이것은 직접 수행하여 성능을 비교함으로서 판단 할 수 있습니다.

**scikit-learn을 사용한 엘라스틱 넷 정규화**

scikit-learn에서는 엘라스틱 넷 정규화는 `ElasticNet` class를 사용하여 수행할 수 있습니다.

`ElasticNet` class는 기존에 사용하던 `LinearRegression` class와 거의 흡사합니다.

`ElasticNet` class는 초기화 시, $\lambda$ 값을 의미하는 `alpha`값과 라쏘의 비율을 의미하는 `l1_ratio`을 설정하여야 합니다.

##### <예제 2> 엘라스틱 넷 정규화

`2-2. <예제 4>` 에서와 같은 데이터를 사용하여 엘라스틱 넷 정규화를 수행하고 loss 값을 출력해 봅시다.

In [5]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet

# 학습용 데이터
feature_data = np.array([1, 2, 3, 4]).reshape((-1,1))
label_data = np.array([3.1,4.9,7.2,8.9]).reshape((-1,1))

# 4차항 선형 회귀에 해당되는 4를 초기값으로 입력
poly = PolynomialFeatures(4)

# fit 함수를 사용하여 X로 변환 작업을 수행
X = poly.fit_transform(feature_data)
print("X: \n{}\n".format(X))

# 릿지 모델 설정 알파 값을 바꿔가며 수행
elastic_model = ElasticNet(alpha=1, l1_ratio=0.5)

# 학습 수행
elastic_model.fit(X, label_data)

# scikit-learn 에서는 loss 함수가 모델안에 내정되어 있지 않기에 정의
def loss(prediction, label):
        
    error = label - prediction
    ls = np.mean(error**2)

    return ls

print("loss: {}\n".format(loss(elastic_model.predict(X), label_data))) 

X: 
[[  1.   1.   1.   1.   1.]
 [  1.   2.   4.   8.  16.]
 [  1.   3.   9.  27.  81.]
 [  1.   4.  16.  64. 256.]]

loss: 9.274181627963248



위 실습에서 `alpha, l1_ratio`값을 바꿔가며 loss 값이 어떻게 바뀌는지 수행해 봅시다.

---